In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)



for y in ['20200709']:
    print('----------------------------------------------------------------')
    print(y)
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zt_88_03_day_pcap\\mdL2Pcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    logSH1["StockID"] = logSH1["ID"] - 1000000
    logSH1 = logSH1[["sequenceNo", "StockID", "time", "cum_volume", "cum_amount", "close",
                     "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q", "bid3q", 
                     "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", 
                     "ask2q", "ask3q", "ask4q", "ask5q", "open", "cum_tradesCnt"]]
    logSH1 = logSH1.rename(columns={"open":"openPrice", "cum_tradesCnt":"numTrades"})
        
    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SH_*'))[0]
    path2 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SZ_*'))[0]
    path3 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zt_96_04_day_96data\\mdLog_SH_*'))[0]
    
    
    startTm = datetime.datetime.now()
    logSH2 = pd.read_csv(path3,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
    print(datetime.datetime.now() - startTm)
    logSH2["time"] = logSH2["time"].apply(lambda x: int((x.replace(':', "")).replace(".", ""))) 
    
    print('----------------------------------------------------------------')
    print('SH lv2 data:')
    in_dex = [16, 300, 852, 905]
    data1 = logSH2[~logSH2["StockID"].isin(in_dex) & (logSH2["time"] >= 91500000) & (logSH2["time"] <= 150000000) \
              & (logSH2["source"] == 13)]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["time"] >= 91500000) & (logSH1["time"] <= 150000000)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "time", "numTrades"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))
    
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    
    for cols in ["cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p","ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "openPrice"]:
        data1_1[cols] = (data1_1[cols]*10000).round(0)
    
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
        display(len(test[np.isnan(test["sequenceNo_y"])])/n1)
        display(len(test[np.isnan(test["sequenceNo_y"])]["time"].unique()))
        display(test[np.isnan(test["sequenceNo_y"])]["time"].unique())
        display(len(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
        display(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["sequenceNo_x"])])
        display(n2-n1)
        print((n2-n1)/n1)
    del logSH2
    del data1
    del data2
    del test
    del data1_1
    del data2_1
    
    
    print('----------------------------------------------------------------')
    print('SH index data:')
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zt_88_03_day_pcap\\mdIndexPcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    index = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    index["StockID"] = index["ID"] - 1000000
    index = index.rename(columns={"open":"openPrice"})

    startTm = datetime.datetime.now()
    logSH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
    print(datetime.datetime.now() - startTm)
    logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    
    in_dex = [16, 300, 852, 905]
    index = index[index["StockID"].isin(in_dex)]
    print(index["StockID"].unique())
    
    data1 = logSH[(logSH["StockID"].isin(in_dex)) & (logSH["time"] >= 91500000) & (logSH["time"] <= 150000000)]
    data2 = index[(index["time"] >= 91500000) & (index["time"] <= 150000000)]
    data1['cum_amount'] = data1['cum_amount'].round(1)

    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    
    for cols in ["cum_amount", "close", "openPrice"]:
        data1_1[cols] = (data1_1[cols]*10000).round(0)
        
        
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["sequenceNo_x"])])
    
    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice", "time"]
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["sequenceNo_x"])])
    
    del index
    del logSH
    del data1
    del data2
    del test
    del data1_1
    del data2_1

    
    print('----------------------------------------------------------------')
    print('SZ lv2 data:')
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_96_03_day_pcap\\mdL2Pcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    logSZ1 = logSZ1.loc[:, ["clockAtArrival", "sequenceNo", "ID", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "open", "cum_tradesCnt"]]
    logSZ1 = logSZ1.rename(columns={"open":"openPrice", "cum_tradesCnt":"numTrades"})
    logSZ1["StockID"] = logSZ1["ID"] - 2000000

    startTm = datetime.datetime.now()
    logSZ = pd.read_csv(path2,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
    logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    print(datetime.datetime.now() - startTm)
    
    startTm = datetime.datetime.now()
    data1 = logSZ[(logSZ["time"] >= 91500000) & (logSZ["time"] < 150000000) & (logSZ["source"] == 4)]
    data2 = logSZ1[(logSZ1["time"] >= 91500000) & (logSZ1["time"] < 150000000)]

    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades", "time"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))
   
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    for i in ["cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "openPrice"]:
        data1_1[i] = (data1_1[i] * 10000).round(0)

    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    print(datetime.datetime.now() - startTm)
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
        print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
        print(np.sort(test[np.isnan(test["sequenceNo_y"])]["time"].unique()))
        print(len(np.sort(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique())))
        print(np.sort(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["sequenceNo_x"])])
        display(n2-n1)
    del logSZ
    del logSZ1
    del data1
    del data2
    del test
    del data1_1
    del data2_1
    
    
    
    
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_96_03_day_pcap\\mdOrderPcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    OrderLogSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    OrderLogSZ1["SecurityID"] = OrderLogSZ1["ID"] - 2000000
    OrderLogSZ1 = OrderLogSZ1.rename(columns={"time":'TransactTime'})
    

    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdOrderLog_*'))[0]
    OrderLogSZ = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
    OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
    
    OrderLogSZ1["OrderType"] = np.where(OrderLogSZ1["OrderType"] == 2, '2', np.where(
        OrderLogSZ1["OrderType"] == 1, '1', OrderLogSZ1['OrderType']))
    

    display(len(OrderLogSZ["SecurityID"].unique()))
    display(len(OrderLogSZ1["SecurityID"].unique()))
    display(len(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique())))
    display(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique()))


    sl = list(set(OrderLogSZ["SecurityID"].unique()) & set(OrderLogSZ1['SecurityID'].unique()))
    OrderLogSZ = OrderLogSZ[OrderLogSZ["SecurityID"].isin(sl)]
    OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["SecurityID"].isin(sl)]
    print(len(OrderLogSZ["SecurityID"].unique()))
    print(len(OrderLogSZ1["SecurityID"].unique()))
    
    print('----------------------------------------------------------------')
    print('SZ order data:')
    
    columns = ["ApplSeqNum", "TransactTime", "Side", 'OrderType', 'Price', 'OrderQty', "SecurityID"]
    ree = pd.merge(OrderLogSZ, OrderLogSZ1, on=columns, how="outer", validate='one_to_one')
    n1 = ree["sequenceNo_x"].count()
    n2 = ree["sequenceNo_y"].count()
    len1 = len(ree)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(ree[np.isnan(ree["sequenceNo_y"])])
        print(len(ree[np.isnan(ree["sequenceNo_y"])]))
        print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("test is complete, baseline is not complete:")
        display(ree[np.isnan(ree["sequenceNo_x"])])
        print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)
#     del OrderLogSZ
#     del OrderLogSZ1
#     del ree
    
    
    
    
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zt_88_03_day_pcap\\mdTradePcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    
    startTm = datetime.datetime.now()
    SH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    SH1["SecurityID"] = SH1["ID"] - 1000000
    SH1 = SH1.rename(columns={"time":'TransactTime'})

    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdTradeLog_*'))[0]
    SH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    SH = SH[SH["exchId"] == 1]
    
    display(len(SH["SecurityID"].unique()))
    display(len(SH1["SecurityID"].unique()))
    display(len(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique())))
    display(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))

    
    sl = list(set(SH["SecurityID"].unique()) & set(SH1['SecurityID'].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
    print(len(SH["SecurityID"].unique()))
    print(len(SH1["SecurityID"].unique()))

    print(SH1.columns)
    
    print('----------------------------------------------------------------')
    print('SH trade data:')
    
    SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
    SH1["ExecType"] = 'F'
    columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]
    re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo_x"].count()
    n2 = re["sequenceNo_y"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["sequenceNo_y"])])
        print(len(re[np.isnan(re["sequenceNo_y"])]))
        print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo_x"])])
        print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)
    del SH
    del SH1
    del re
    
    
    
    
    
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_96_03_day_pcap\\mdTradePcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    
    startTm = datetime.datetime.now()
    TradeLogSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    TradeLogSZ1["SecurityID"] = TradeLogSZ1["ID"] - 2000000
    TradeLogSZ1 = TradeLogSZ1.rename(columns={"time":'TransactTime'})
    TradeLogSZ1["TradeBSFlag"] = 'N'
    
    startTm = datetime.datetime.now()
    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdTradeLog_*'))[0]

    TradeLogSZ = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    TradeLogSZ = TradeLogSZ[TradeLogSZ["exchId"] != 1]
    print(datetime.datetime.now() - startTm)
    
    
    display(len(TradeLogSZ["SecurityID"].unique()))
    display(len(TradeLogSZ1["SecurityID"].unique()))
    display(len(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique())))
    display(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique()))


    sl = list(set(TradeLogSZ["SecurityID"].unique()) & set(TradeLogSZ1['SecurityID'].unique()))
    TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(sl)]
    TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(sl)]
    print(len(TradeLogSZ["SecurityID"].unique()))
    print(len(TradeLogSZ1["SecurityID"].unique()))

    print(TradeLogSZ1.columns)
    
    
    print('----------------------------------------------------------------')
    print('SZ trade data:')
    
    TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
    TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

    columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
    re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo_x"].count()
    n2 = re["sequenceNo_y"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["sequenceNo_y"])])
        print(len(re[np.isnan(re["sequenceNo_y"])]))
        print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo_x"])])
        print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)
    del TradeLogSZ
    del TradeLogSZ1
    del re

----------------------------------------------------------------
20200709


C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0:01:54.188039


C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2224

3138

0

set()

2224
2224
----------------------------------------------------------------
SZ order data:


In [7]:
logSH[(logSH['StockID'] == 300) & (logSH['cum_volume'] >= 222338014)].head(10)

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
9213479,1594185757968597,121193509,5,300,SH,132232000,222338014,3.400578e+11,4712.6909,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4692.8621
9218259,1594185762452772,121238478,5,300,SH,132236000,222338014,3.400578e+11,4712.6909,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4692.8621
9218512,1594185763043736,121246862,5,300,SH,132237000,222435076,3.401898e+11,4712.3218,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4692.8621
9223215,1594185768067324,121295486,5,300,SH,132242000,222526788,3.403229e+11,4712.1758,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4692.8621
9227692,1594185772979964,121349942,5,300,SH,132247000,222610414,3.404580e+11,4711.9512,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4692.8621
9232665,1594185777947578,121402695,5,300,SH,132252000,222714805,3.406264e+11,4711.4416,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4692.8621
9237368,1594185783067644,121452972,5,300,SH,132257000,222792808,3.407543e+11,4711.0176,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4692.8621
9241872,1594185788019162,121505891,5,300,SH,132302000,222877855,3.408966e+11,4710.7522,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4692.8621
9246584,1594185792533113,121550321,5,300,SH,132306000,222877855,3.408966e+11,4710.7522,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4692.8621
9246798,1594185792978189,121556650,5,300,SH,132307000,222976331,3.410455e+11,4711.5751,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4692.8621


In [6]:
index[(index['StockID'] == 300) & (index['cum_volume'] >= 222338014)].head(5)

,clockAtArrival,sequenceNo,ID,time,cum_volume,cum_amount,close,openPrice,prevClose,StockID
454331,1594185786623968,41855875,1000300,132229110,222338014,3400578202558000,47126909,46928621,46981264,300
454549,1594185791692593,41875233,1000300,132234380,222435076,3401898445726000,47123218,46928621,46981264,300
454957,1594185801643913,41912131,1000300,132244120,222610414,3404579530902000,47119512,46928621,46981264,300
455163,1594185806602534,41930074,1000300,132249390,222714805,3406263504840000,47114416,46928621,46981264,300
455385,1594185811688326,41948028,1000300,132254390,222792808,3407542540782000,47110176,46928621,46981264,300


In [4]:
logSZ1.groupby(['StockID'])['time'].max().min()

103318000

In [16]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)



for y in ['20200707']:
    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SH_*'))[0]

    
    print('----------------------------------------------------------------')
    print('SH index data:')
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zt_88_03_day_pcap\\mdIndexPcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    index = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    index["StockID"] = index["ID"] - 1000000
    index = index.rename(columns={"open":"openPrice"})

    startTm = datetime.datetime.now()
    logSH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
    print(datetime.datetime.now() - startTm)
    logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    
    in_dex = [16, 300, 852, 905]
    index = index[index["StockID"].isin(in_dex)]
    print(index["StockID"].unique())
    
    data1 = logSH[(logSH["StockID"].isin(in_dex)) & (logSH["time"] >= 91500000) & (logSH["time"] <= 150000000)]
    data2 = index[(index["time"] >= 91500000) & (index["time"] <= 150000000)]
    data1['cum_amount'] = data1['cum_amount'].round(1)
    
    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    
    for cols in ["cum_amount", "close", "openPrice"]:
        data1_1[cols] = (data1_1[cols]*10000).round(0)
        
        
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["sequenceNo_x"])])
    
    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice", "time"]
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["sequenceNo_x"])])

----------------------------------------------------------------
SH index data:
0:00:01.326454
0:01:18.822090
[852 300 905  16]
11402
11406
11406


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'baseline is not complete::'

,index_x,clockAtArrival_x,sequenceNo_x,source,StockID,exchange,time_x,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,index_y,clockAtArrival_y,sequenceNo_y,ID,time_y,prevClose
11402,NaN,NaN,NaN,NaN,16,NaN,NaN,121403365,2.016989e+15,33797576.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31987384.0,687540,1594018834882949,65483928,1000016,145959400,31591583
11403,NaN,NaN,NaN,NaN,300,NaN,NaN,402943123,5.931332e+15,46744303.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44658081.0,687668,1594018834947643,65484271,1000300,145959400,44195955
11404,NaN,NaN,NaN,NaN,852,NaN,NaN,301838202,3.286361e+15,67258452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65169221.0,687678,1594018834965077,65484329,1000852,145959400,64783539
11405,NaN,NaN,NaN,NaN,905,NaN,NaN,299765040,2.891759e+15,63000949.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60959561.0,687682,1594018834965077,65484333,1000905,145959400,60515099


11402
11406
22808


'test is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,index_y,clockAtArrival_y,sequenceNo_y,ID,prevClose
0,43910.0,1.593998e+15,396341.0,5.0,16,SH,91502000,0,0.000000e+00,31591583.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,43911.0,1.593998e+15,401628.0,5.0,300,SH,91502000,0,0.000000e+00,44190231.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,43912.0,1.593998e+15,401653.0,5.0,852,SH,91502000,0,0.000000e+00,64778880.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,43913.0,1.593998e+15,402460.0,5.0,905,SH,91502000,0,0.000000e+00,60504248.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,314138.0,1.593999e+15,2297844.0,5.0,16,SH,92522000,1300880,2.547593e+13,31987384.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31987384.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11397,14707060.0,1.594019e+15,181271636.0,5.0,905,SH,145702000,299588285,2.889607e+15,63001561.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60959561.0,NaN,NaN,NaN,NaN,NaN
11398,14712629.0,1.594019e+15,181299314.0,5.0,16,SH,145707000,121002907,2.010341e+15,33797576.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31987384.0,NaN,NaN,NaN,NaN,NaN
11399,14712630.0,1.594019e+15,181299564.0,5.0,300,SH,145707000,402228920,5.921911e+15,46744303.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44658081.0,NaN,NaN,NaN,NaN,NaN
11400,14712631.0,1.594019e+15,181299602.0,5.0,852,SH,145707000,301683073,3.284478e+15,67258452.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65169221.0,NaN,NaN,NaN,NaN,NaN


In [9]:
data1['cum_amount'][data1['cum_amount'].astype(str).apply(lambda x: len(str(x.split('.')[1]))) == 1].iloc[10000]

264566283391.6

In [13]:
data1[(data1["StockID"] == 300)].tail(5)

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
14845524,1594018782635193,181917922,5,300,SH,145937000,402228920,5.921911e+11,4674.4303,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4465.8081
14849519,1594018787602377,181932072,5,300,SH,145942000,402228920,5.921911e+11,4674.4303,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4465.8081
14853531,1594018792571940,181950217,5,300,SH,145947000,402228920,5.921911e+11,4674.4303,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4465.8081
14857936,1594018797632293,181966906,5,300,SH,145952000,402228920,5.921911e+11,4674.4303,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4465.8081
14861900,1594018802603228,181982431,5,300,SH,145957000,402228920,5.921911e+11,4674.4303,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,4465.8081


In [14]:
data2[(data2["StockID"] == 300)].tail(5)

,clockAtArrival,sequenceNo,ID,time,cum_volume,cum_amount,close,openPrice,prevClose,StockID
686518,1594018789092405,65401151,1000300,145939840,402228920,5921910824551000,46744303,44658081,44195955,300
686707,1594018794062513,65410748,1000300,145944430,402228920,5921910824551000,46744303,44658081,44195955,300
686943,1594018799122806,65420718,1000300,145949740,402228920,5921910824551000,46744303,44658081,44195955,300
687153,1594018804093061,65430445,1000300,145954810,402228920,5921910824551000,46744303,44658081,44195955,300
687350,1594018809063405,65456843,1000300,145959400,402943123,5931332270391000,46744303,44658081,44195955,300


In [10]:
data1_1[(data1_1["StockID"] == 300) & (data1_1['cum_volume'] == 299381612)]

,index,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
7615,10173336,1594013902684189,133907189,5,300,SH,133817000,299381612,4.504192e+15,46354412.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,44658081.0
7618,10173442,1594013902800052,133908137,5,300,SH,133817000,299381612,4.504192e+15,46354412.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,44658081.0


In [7]:
data2_1[(data2_1["StockID"] == 300) & (data2_1['cum_volume'] == 299381612)]

,index,clockAtArrival,sequenceNo,ID,time,cum_volume,cum_amount,close,openPrice,prevClose,StockID
7615,477388,1594013904074748,47849040,1000300,133814800,299381612,4504191665621000,46354412,44658081,44195955,300


In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)



for y in ['20200706']:
    print('----------------------------------------------------------------')
    print(y)
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zt_88_03_day_pcap\\mdL2Pcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    logSH1["StockID"] = logSH1["ID"] - 1000000
    logSH1 = logSH1[["sequenceNo", "StockID", "time", "cum_volume", "cum_amount", "close",
                     "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q", "bid3q", 
                     "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", 
                     "ask2q", "ask3q", "ask4q", "ask5q", "open", "cum_tradesCnt"]]
    logSH1 = logSH1.rename(columns={"open":"openPrice", "cum_tradesCnt":"numTrades"})
        
    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SH_*'))[0]
    path2 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SZ_*'))[0]
    path3 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zt_96_04_day_96data\\mdLog_SH_*'))[0]
    
    
    startTm = datetime.datetime.now()
    logSH2 = pd.read_csv(path3,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
    print(datetime.datetime.now() - startTm)
    logSH2["time"] = logSH2["time"].apply(lambda x: int((x.replace(':', "")).replace(".", ""))) 
    
    print('----------------------------------------------------------------')
    print('SH lv2 data:')
    in_dex = [16, 300, 852, 905]
    data1 = logSH2[~logSH2["StockID"].isin(in_dex) & (logSH2["time"] >= 91500000) & (logSH2["time"] <= 150000000) \
              & (logSH2["source"] == 13)]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["time"] >= 91500000) & (logSH1["time"] <= 150000000)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "time", "numTrades"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))
    
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    
    for cols in ["cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p","ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "openPrice"]:
        data1_1[cols] = (data1_1[cols]*10000).round(0)
    
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
        display(len(test[np.isnan(test["sequenceNo_y"])])/n1)
        display(len(test[np.isnan(test["sequenceNo_y"])]["time"].unique()))
        display(test[np.isnan(test["sequenceNo_y"])]["time"].unique())
        display(len(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
        display(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["sequenceNo_x"])])
        display(n2-n1)
        print((n2-n1)/n1)
    del logSH2
    del data1
    del data2
    del test
    del data1_1
    del data2_1
    
    
    print('----------------------------------------------------------------')
    print('SH index data:')
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zt_88_03_day_pcap\\mdIndexPcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    index = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    index["StockID"] = index["ID"] - 1000000
    index = index.rename(columns={"open":"openPrice"})

    startTm = datetime.datetime.now()
    logSH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
    print(datetime.datetime.now() - startTm)
    logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    
    in_dex = [16, 300, 852, 905]
    index = index[index["StockID"].isin(in_dex)]
    print(index["StockID"].unique())
    
    data1 = logSH[(logSH["StockID"].isin(in_dex)) & (logSH["time"] >= 91500000) & (logSH["time"] <= 150000000)]
    data2 = index[(index["time"] >= 91500000) & (index["time"] <= 150000000)]

    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    
    for cols in ["cum_amount", "close", "openPrice"]:
        data1_1[cols] = (data1_1[cols]*10000).round(0)
        
        
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["sequenceNo_x"])])
    
    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice", "time"]
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["sequenceNo_x"])])
    
    del index
    del logSH
    del data1
    del data2
    del test
    del data1_1
    del data2_1

    
    print('----------------------------------------------------------------')
    print('SZ lv2 data:')
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_88_04_day_pcap\\mdL2Pcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    logSZ1 = logSZ1.loc[:, ["clockAtArrival", "sequenceNo", "ID", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "open", "cum_tradesCnt"]]
    logSZ1 = logSZ1.rename(columns={"open":"openPrice", "cum_tradesCnt":"numTrades"})
    logSZ1["StockID"] = logSZ1["ID"] - 2000000

    startTm = datetime.datetime.now()
    logSZ = pd.read_csv(path2,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
    logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    print(datetime.datetime.now() - startTm)
    
    startTm = datetime.datetime.now()
    data1 = logSZ[(logSZ["time"] >= 91500000) & (logSZ["time"] < 150000000) & (logSZ["source"] == 4)]
    data2 = logSZ1[(logSZ1["time"] >= 91500000) & (logSZ1["time"] < 150000000)]

    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades", "time"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))
   
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    for i in ["cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "openPrice"]:
        data1_1[i] = (data1_1[i] * 10000).round(0)

    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    print(datetime.datetime.now() - startTm)
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
        print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
        print(np.sort(test[np.isnan(test["sequenceNo_y"])]["time"].unique()))
        print(len(np.sort(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique())))
        print(np.sort(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["sequenceNo_x"])])
        display(n2-n1)
    del logSZ
    del logSZ1
    del data1
    del data2
    del test
    del data1_1
    del data2_1
    
    
    
    
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_88_04_day_pcap\\mdOrderPcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    OrderLogSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    OrderLogSZ1["SecurityID"] = OrderLogSZ1["ID"] - 2000000
    OrderLogSZ1 = OrderLogSZ1.rename(columns={"time":'TransactTime'})
    

    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdOrderLog_*'))[0]
    OrderLogSZ = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
    OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
    
    OrderLogSZ1["OrderType"] = np.where(OrderLogSZ1["OrderType"] == 2, '2', np.where(
        OrderLogSZ1["OrderType"] == 1, '1', OrderLogSZ1['OrderType']))
    

    display(len(OrderLogSZ["SecurityID"].unique()))
    display(len(OrderLogSZ1["SecurityID"].unique()))
    display(len(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique())))
    display(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique()))


    sl = list(set(OrderLogSZ["SecurityID"].unique()) & set(OrderLogSZ1['SecurityID'].unique()))
    OrderLogSZ = OrderLogSZ[OrderLogSZ["SecurityID"].isin(sl)]
    OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["SecurityID"].isin(sl)]
    print(len(OrderLogSZ["SecurityID"].unique()))
    print(len(OrderLogSZ1["SecurityID"].unique()))
    
    print('----------------------------------------------------------------')
    print('SZ order data:')
    
    columns = ["ApplSeqNum", "TransactTime", "Side", 'OrderType', 'Price', 'OrderQty', "SecurityID"]
    ree = pd.merge(OrderLogSZ, OrderLogSZ1, on=columns, how="outer", validate='one_to_one')
    n1 = ree["sequenceNo_x"].count()
    n2 = ree["sequenceNo_y"].count()
    len1 = len(ree)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(ree[np.isnan(ree["sequenceNo_y"])])
        print(len(ree[np.isnan(ree["sequenceNo_y"])]))
        print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("test is complete, baseline is not complete:")
        display(ree[np.isnan(ree["sequenceNo_x"])])
        print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)
    del OrderLogSZ
    del OrderLogSZ1
    del ree
    
    
    
    
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zt_88_03_day_pcap\\mdTradePcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    
    startTm = datetime.datetime.now()
    SH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    SH1["SecurityID"] = SH1["ID"] - 1000000
    SH1 = SH1.rename(columns={"time":'TransactTime'})

    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdTradeLog_*'))[0]
    SH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    SH = SH[SH["exchId"] == 1]
    
    display(len(SH["SecurityID"].unique()))
    display(len(SH1["SecurityID"].unique()))
    display(len(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique())))
    display(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))

    
    sl = list(set(SH["SecurityID"].unique()) & set(SH1['SecurityID'].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
    print(len(SH["SecurityID"].unique()))
    print(len(SH1["SecurityID"].unique()))

    print(SH1.columns)
    
    print('----------------------------------------------------------------')
    print('SH trade data:')
    
    SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
    SH1["ExecType"] = 'F'
    columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]
    re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo_x"].count()
    n2 = re["sequenceNo_y"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["sequenceNo_y"])])
        print(len(re[np.isnan(re["sequenceNo_y"])]))
        print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo_x"])])
        print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)
    del SH
    del SH1
    del re
    
    
    
    
    
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_88_04_day_pcap\\mdTradePcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    
    startTm = datetime.datetime.now()
    TradeLogSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    TradeLogSZ1["SecurityID"] = TradeLogSZ1["ID"] - 2000000
    TradeLogSZ1 = TradeLogSZ1.rename(columns={"time":'TransactTime'})
    TradeLogSZ1["TradeBSFlag"] = 'N'
    
    startTm = datetime.datetime.now()
    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdTradeLog_*'))[0]

    TradeLogSZ = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    TradeLogSZ = TradeLogSZ[TradeLogSZ["exchId"] != 1]
    print(datetime.datetime.now() - startTm)
    
    
    display(len(TradeLogSZ["SecurityID"].unique()))
    display(len(TradeLogSZ1["SecurityID"].unique()))
    display(len(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique())))
    display(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique()))


    sl = list(set(TradeLogSZ["SecurityID"].unique()) & set(TradeLogSZ1['SecurityID'].unique()))
    TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(sl)]
    TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(sl)]
    print(len(TradeLogSZ["SecurityID"].unique()))
    print(len(TradeLogSZ1["SecurityID"].unique()))

    print(TradeLogSZ1.columns)
    
    
    print('----------------------------------------------------------------')
    print('SZ trade data:')
    
    TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
    TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

    columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
    re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo_x"].count()
    n2 = re["sequenceNo_y"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["sequenceNo_y"])])
        print(len(re[np.isnan(re["sequenceNo_y"])]))
        print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo_x"])])
        print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)
    del TradeLogSZ
    del TradeLogSZ1
    del re

----------------------------------------------------------------
20200706
0:05:42.404404
0:01:41.178653
----------------------------------------------------------------
SH lv2 data:
1616
11964
0
set()
7275470
7282377
7282377
-----------------------------------------------


'baseline is not complete:'

,index_x,clockAtArrival,sequenceNo_x,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades,index_y,sequenceNo_y
7275470,NaN,NaN,NaN,NaN,601099,NaN,91500000,0,0.0,0.0,42600.0,0.0,0.0,0.0,0.0,69500,400400,0,0,0,42600.0,0.0,0.0,0.0,0.0,69500,0,0,0,0,0.0,0,202410,637669
7275471,NaN,NaN,NaN,NaN,601816,NaN,91500000,0,0.0,0.0,65400.0,0.0,0.0,0.0,0.0,100,0,0,0,0,65400.0,0.0,0.0,0.0,0.0,100,200,0,0,0,0.0,0,202411,637670
7275472,NaN,NaN,NaN,NaN,600109,NaN,91500000,0,0.0,0.0,153500.0,0.0,0.0,0.0,0.0,200,95900,0,0,0,153500.0,0.0,0.0,0.0,0.0,200,0,0,0,0,0.0,0,202412,637671
7275473,NaN,NaN,NaN,NaN,600221,NaN,91500000,0,0.0,0.0,16300.0,0.0,0.0,0.0,0.0,260000,0,0,0,0,16300.0,0.0,0.0,0.0,0.0,260000,69900,0,0,0,0.0,0,202413,637672
7275474,NaN,NaN,NaN,NaN,601229,NaN,91500000,0,0.0,0.0,89100.0,0.0,0.0,0.0,0.0,100,500,0,0,0,89100.0,0.0,0.0,0.0,0.0,100,0,0,0,0,0.0,0,202414,637673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7282372,NaN,NaN,NaN,NaN,601198,NaN,91539000,0,0.0,0.0,129800.0,0.0,0.0,0.0,0.0,44300,0,0,0,0,129800.0,0.0,0.0,0.0,0.0,44300,31100,0,0,0,0.0,0,218805,682332
7282373,NaN,NaN,NaN,NaN,601866,NaN,91539000,0,0.0,0.0,20200.0,0.0,0.0,0.0,0.0,165700,0,0,0,0,20200.0,0.0,0.0,0.0,0.0,165700,51900,0,0,0,0.0,0,219329,683281
7282374,NaN,NaN,NaN,NaN,601899,NaN,91539000,0,0.0,0.0,47500.0,0.0,0.0,0.0,0.0,320800,0,0,0,0,47500.0,0.0,0.0,0.0,0.0,320800,1006526,0,0,0,0.0,0,219330,683282
7282375,NaN,NaN,NaN,NaN,600556,NaN,91540000,0,0.0,0.0,177500.0,0.0,0.0,0.0,0.0,19000,47800,0,0,0,177500.0,0.0,0.0,0.0,0.0,19000,0,0,0,0,0.0,0,219344,683296


6907

0.0009493544746937312
----------------------------------------------------------------
SH index data:
0:00:01.944751
0:01:48.464746
[852 300 905  16]
11848
11406
12333


...

2212

3111

0

set()

2212
2212
Index(['clockAtArrival', 'sequenceNo', 'ID', 'TransactTime', 'ApplSeqNum',
       'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum',
       'OfferApplSeqNum', 'SecurityID', 'TradeBSFlag'],
      dtype='object')
----------------------------------------------------------------
SZ trade data:
55693579
55693579
55693579
-----------------------------------------------


In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)

for y in ['20200703']:
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_96_03_day_pcap\\mdOrderPcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    OrderLogSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    OrderLogSZ1["SecurityID"] = OrderLogSZ1["ID"] - 2000000
    OrderLogSZ1 = OrderLogSZ1.rename(columns={"time":'TransactTime'})
    

    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdOrderLog_*'))[0]
    OrderLogSZ = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
    OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
    
    OrderLogSZ1["OrderType"] = np.where(OrderLogSZ1["OrderType"] == 2, '2', np.where(
        OrderLogSZ1["OrderType"] == 1, '1', OrderLogSZ1['OrderType']))
    

    display(len(OrderLogSZ["SecurityID"].unique()))
    display(len(OrderLogSZ1["SecurityID"].unique()))
    display(len(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique())))
    display(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique()))


    sl = list(set(OrderLogSZ["SecurityID"].unique()) & set(OrderLogSZ1['SecurityID'].unique()))
    OrderLogSZ = OrderLogSZ[OrderLogSZ["SecurityID"].isin(sl)]
    OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["SecurityID"].isin(sl)]
    print(len(OrderLogSZ["SecurityID"].unique()))
    print(len(OrderLogSZ1["SecurityID"].unique()))
    
    print('----------------------------------------------------------------')
    print('SZ order data:')
    
    columns = ["ApplSeqNum", "TransactTime", "Side", 'OrderType', 'Price', 'OrderQty', "SecurityID"]
    ree = pd.merge(OrderLogSZ, OrderLogSZ1, on=columns, how="outer", validate='one_to_one')
    n1 = ree["sequenceNo_x"].count()
    n2 = ree["sequenceNo_y"].count()
    len1 = len(ree)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(ree[np.isnan(ree["sequenceNo_y"])])
        print(len(ree[np.isnan(ree["sequenceNo_y"])]))
        print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("test is complete, baseline is not complete:")
        display(ree[np.isnan(ree["sequenceNo_x"])])
        print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)


0:01:33.867848


2211

3129

0

set()

2211
2211
----------------------------------------------------------------
SZ order data:
50861230
50861205
50861301
-----------------------------------------------


'test is not complete:'

,clockAtArrival_x,sequenceNo_x,exchId,TransactTime,ApplSeqNum,SecurityID,Side,OrderType,Price,OrderQty,clockAtArrival_y,sequenceNo_y,ID
14514481,1.593741e+15,37057921.0,2.0,95640180,6568381,2030,2,2,268700,8200,NaN,NaN,NaN
14514482,1.593741e+15,37057923.0,2.0,95640180,6568383,2619,1,2,25500,5000,NaN,NaN,NaN
14514483,1.593741e+15,37057924.0,2.0,95640180,6400591,2905,2,2,105500,100,NaN,NaN,NaN
14514484,1.593741e+15,37057925.0,2.0,95640180,6400592,2583,2,2,81500,600,NaN,NaN,NaN
14514485,1.593741e+15,37057926.0,2.0,95640180,6462880,2841,2,2,963400,300,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14514653,1.593741e+15,37058397.0,2.0,95640210,6425607,300174,1,2,188600,100,NaN,NaN,NaN
14514656,1.593741e+15,37058403.0,2.0,95640210,6568485,300149,2,2,200100,100,NaN,NaN,NaN
14514661,1.593741e+15,37058413.0,2.0,95640210,6462982,300166,2,2,132200,20000,NaN,NaN,NaN
14514662,1.593741e+15,37058415.0,2.0,95640210,6462984,300740,1,2,153400,800,NaN,NaN,NaN


96
[95640180 95640190 95640200 95640210]
81
[  2030   2619   2905   2583   2841    538 300085   2123   2425   2891
 300236   2317 300101    100   2415    830   2187    717   2456 300406
   2049    403 300618   2515   2174     63 300368   2643   2960 300738
    563 300820      1   2437 300377    905    887   2475   2902   2281
    576 300058   2429   2110   2466    908    592   2195   2074    786
 300123    783 300059   2104 300677 300174   2407   2610   2607 300159
   2214   2497 300748 300655   2807   2125   2666   2839   2959   2383
   2460   2612 300045 300235   2714   2261    700 300166 300740 300034
 300149]


In [6]:
print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())

[145959960 145959970 145959980 145959990]
67
[300253   2165   2398    333   2493 300009     58   2429   2571 300726
 300453   2475   2023 300476    516    776 300232   2498   2065   2389
    750   2839 300274   2465 300079   2631    716   2001   2625   1872
      9    510    513 300102 300189   2388 300272   2584 300059   2384
   2460    691   2239 300203 300171   2400 300541      1    592   2184
    999   2594   2348   2531   2092   2020 300305   2661   2602    676
   2024   2757    977   2074 300782   2041   2517]


In [4]:
OrderLogSZ[(OrderLogSZ["SecurityID"] == 2030) & (OrderLogSZ["ApplSeqNum"] == 6568381)]

,clockAtArrival,sequenceNo,exchId,TransactTime,ApplSeqNum,SecurityID,Side,OrderType,Price,OrderQty
14514481,1593741404937162,37057921,2,95640180,6568381,2030,2,2,268700,8200


In [5]:
OrderLogSZ1[(OrderLogSZ1["SecurityID"] == 2030) & (OrderLogSZ1["ApplSeqNum"] == 6568381)]

,clockAtArrival,sequenceNo,ID,TransactTime,ApplSeqNum,Side,OrderType,Price,OrderQty,SecurityID


In [9]:
logSZ1[(logSZ1["StockID"] == 2030) & (logSZ1["time"] > 103200000)][["StockID", "time", "cum_volume", "close", "bid1p", "bid2p", "ask1p", "ask2p", "bid1q", "bid2q", "ask1q", "ask2q"]].head(5)

,StockID,time,cum_volume,close,bid1p,bid2p,ask1p,ask2p,bid1q,bid2q,ask1q,ask2q
3195504,2616,103203000,2702600,116800,116800,116600,117000,117200,2100,31900,4200,700
3199510,2616,103209000,2706300,116600,116600,116500,116700,116800,30600,6700,2100,2300
3201480,2616,103212000,2707000,116700,116600,116500,116700,116800,30600,6700,3500,2300
3203381,2616,103215000,2707100,116700,116600,116500,116700,116800,30600,6700,4100,2300
3205322,2616,103218000,2707100,116700,116600,116500,116700,116800,30600,6700,4200,2300


In [8]:
logSZ[(logSZ["StockID"] == 2030) & (logSZ["time"] > 103200000) & (logSZ["source"] == 4)][["StockID", "time", "cum_volume", "close", "bid1p", "bid2p", "ask1p", "ask2p", "bid1q", "bid2q", "ask1q", "ask2q"]].head(5)

,StockID,time,cum_volume,close,bid1p,bid2p,ask1p,ask2p,bid1q,bid2q,ask1q,ask2q
4705606,2616,103203000,2702600,11.68,11.68,11.66,11.70,11.72,2100,31900,4200,700
4709176,2616,103206000,2704700,11.68,11.67,11.66,11.68,11.69,300,31900,2300,700
4712611,2616,103209000,2706300,11.66,11.66,11.65,11.67,11.68,30600,6700,2100,2300
4716071,2616,103212000,2707000,11.67,11.66,11.65,11.67,11.68,30600,6700,3500,2300
4719401,2616,103215000,2707100,11.67,11.66,11.65,11.67,11.68,30600,6700,4100,2300


In [5]:
readPath = r'\\mentos\dailyRawData\logs_20200618_zt_52_03_day_pcap\***'
dataPathLs = np.array(glob.glob(readPath))
dataPathLs

array([], dtype=float64)